* Họ tên: Phạm Thị Linh 
* Mssv: 18020769 
* Mã lớp: INT3405 1

# **1. Đặt vấn đề**
Quora là một nền tảng cho phép mọi người học hỏi lẫn nhau. Trên Quora, mọi người có thể đặt câu hỏi và kết nối với những người khác, những người đóng góp thông tin chi tiết độc đáo và câu trả lời chất lượng. Một thách thức quan trọng là loại bỏ những câu hỏi thiếu chân thành - những câu hỏi được đặt ra dựa trên những tiền đề sai lầm hoặc có ý định đưa ra một tuyên bố hơn là tìm kiếm những câu trả lời hữu ích.

Trong cuộc thi này, Kagglers sẽ phát triển các mô hình xác định và gắn cờ cho các câu hỏi không chân thành.

**Mô tả bài toán**

* Đầu vào : dữ liệu các câu hỏi dưới dạng text
* Đầu ra : phân loại được câu hỏi là insincere hay sincere

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from subprocess import check_output

%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import os
import gc
import csv
import re
import string

from tqdm import tqdm
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
from scipy.sparse import hstack
from IPython.display import Image
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.util import ngrams

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# **2. Đọc và phân tích dữ liệu**
**2.1. Đọc dữ liệu**

Dữ liệu được cung cấp gồm 2 file train.csv và test.csv

In [ ]:
#Đọc dữ liệu từ file data
train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
#Đọc dữ liệu từ file test 
test=pd.read_csv("../input/quora-insincere-questions-classification/test.csv")

**2.2. Phân tích dữ liệu**

In [ ]:
train

Dữ liệu:
* 1306122 hàng × 3 cột
* qid: id độc nhất của câu hỏi, qid chắc sẽ không tham gia vào bước phân lớp câu hỏi nên có thể bỏ được
* question_text: dữ liệu câu hỏi, do trường này là duy nhất tác động trực tiếp vào phân lớp của câu hỏi nên cần phải thực hiện tiền xử lý
* target: phân lớp của câu hỏi, target = 0 với câu hỏi sincere và target = 1 với câu hỏi incinsere

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
print("Train shape :", train.shape)
print("Test shape :", test.shape)

> Tập dữ liệu train có 1306122 câu hỏi và tập test chứa 375806 câu hỏi để đánh giá hiệu năng mô hình. Trong đó, không có dữ liệu nào bất thường (missing, null) trong cả hai tập.

In [ ]:
train.groupby("target")['qid'].count().plot.bar()

In [ ]:
toxic = train[train.target == 1]
non_toxic = train[train.target == 0]
print(toxic.shape, non_toxic.shape)

In [ ]:
print('%Insincere Questions : {}%'.format(round(train['target'].mean()*100, 2)))

> Nhận xét: số lượng câu hỏi sincerce-insincere có sự chênh lệch rất lớn:
> 
> * Số lượng câu hỏi sincerce chiếm đến gần 94% tập dữ liệu (1225312 câu)
> * Số lượng câu hỏi insincere chiếm 6.2% tập dữ liệu (80810 câu)
> 
> -> Dữ liệu này mất cân bằng rất lớn
> 
> -> Cần áp dụng các phương pháp cân bằng dữ liệu, chia lại tập dữ liệu sao cho câu hỏi toxic và không toxic trở nên cân bằng hơn.

**2.3. Xử lí mất cân bằng dữ liệu**

Hai cách tiếp cận để tạo ra một tập dữ liệu cân bằng trong một tập dữ liệu không cân bằng là under-sampling và oversampling.
* Under-sampling: việc lấy mẫu làm cân bằng tập dữ liệu bằng cách giảm kích thước của lớp trội. Phương pháp này được sử dụng khi số lượng dữ liệu là đủ. Bằng cách giữ tất cả các mẫu trong lớp hiếm và chọn ngẫu nhiên một số trong lớp trội, một tập dữ liệu cân bằng mới có thể được lấy ra để lập mô hình tiếp theo.
* Over-sampling: ngược lại, oversampling được sử dụng khi số lượng dữ liệu không đủ. Nó cố gắng cân bằng số liệu bằng cách tăng kích thước của các mẫu hiếm. Thay vì loại bỏ các mẫu phong phú, các mẫu hiếm mới được tạo ra.

In [ ]:
from sklearn.utils import resample
sincere = train[train.target == 0]
insincere = train[train.target == 1]
# Tỉ lệ 4:1 cho kết quả tốt nhất
x = pd.concat([resample(sincere,replace = True,n_samples = len(insincere)*4), insincere])

In [ ]:
y = x['target']
y.value_counts().plot(kind='bar', rot=0)

* Sau khi chia lại dữ liệu ta thấy tập dữ liệu đã cân bằng hơn trước

# **3. Xử lý dữ liệu**
**3.1. Chuẩn bị dữ liệu**

* Word tokenization: phân tách các từ trong câu là bước cần thiết nhất để xử lý văn bản. Nó giúp tách các câu thành danh sách các từ được phân tách bằng dấu phẩy một cách hiệu quả.
* Loại bỏ stopword (các từ thêm nghĩa cho câu như a, an, the,... ): do không có giá trị khai thác nên cần loại bỏđể không ảnh hưởng tới mô hình
* Tìm từ gốc: từ gốc là từ cấu thành nên các từ khác. VD: Do là từ gốc tạo ra các từ do, does, doing,... Có 2 cách tiếp cận là dùng Lemmatization và Stemming.
    * Lemmatization : là một phương pháp nhóm các từ thành một thuật ngữ duy nhất dựa trên sự biến đổi của chúng.
    * Stemming : là phương pháp khác được sử dụng để liên kết các từ với các từ gốc. Đó là quá trình xác định các biến thể của các từ cơ sở.
> Cả hai cách tiếp cận này đều hiệu quả trong project này nên chúng ta sẽ sử dụng cả hai.

* Text Vectorization giúp chuyển đổi văn bản dữ liệu thành một chuỗi các số để xử lý. Hai cách tiếp cận phổ biến nhất là TF-IDF và Bag of Words.
* Chia tập dữ liệu: tập dữ liệu sẽ được chia theo tỉ lệ 80:20, ứng với tập train và tập test
* Modeling: mô hình được sử dụng cho bài toán này là Logistic Regression và Gradient Boosting

In [ ]:
#loại bỏ các từ trong stopword
stopword_list = nltk.corpus.stopwords.words('english')
def remove_stopwords(text, is_lower_case=True):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
#Stemming
from nltk.stem import  SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
def stem_text(text):
    tokenizer = ToktokTokenizer()
    stemmer = SnowballStemmer('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

In [ ]:
#Lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.toktok import ToktokTokenizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemma_text(text):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

**3.2. Xử lí dữ liệu**

Qua phân tích ở trên ta thấy dữ liệu có có những tác nhân gây nhiễu hoặc không cần thiết , dư thừa để xác nhận xem câu hỏi có phải toxic hay không . Qua đó chúng ta sẽ tìm cách loại bỏ nó :
* Loại bỏ những kí tự đặc biệt có trong câu
* Loại bỏ dấu câu
* Loại bỏ chữ số
* Thay thế những từ sai chính tả
* Thay thế các từ viết tắt
* Loại bỏ những từ là stop word ( ví dụ "a" , "an", "the" ,...)
* Biến đổi 1 từ về dạng gốc (được gọi là stem hoặc root form) bằng cách loại bỏ 1 số ký tự nằm ở cuối từ mà nó nghĩ rằng là biến thể của từ giống như trên nhưng xử lý bằng cách loại bỏ các ký tự cuối từ theo thuật toán heuristic (lemmatization)

In [ ]:
puncts=[',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', 
        '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', 
        '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']
#loại bỏ kí tự đặc biệt
def clean_punct(text):
    for punct in puncts:
        if punct in text:
            text = text.replace(punct, '{}' .format(punct))
    return text

In [ ]:
#loại bỏ chữ số 
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'bitcoin', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization', 
                'electroneum':'bitcoin','nanodegree':'degree','hotstar':'star','dream11':'dream','ftre':'fire','tensorflow':'framework','unocoin':'bitcoin',
                'lnmiit':'limit','unacademy':'academy','altcoin':'bitcoin','altcoins':'bitcoin','litecoin':'bitcoin','coinbase':'bitcoin','cryptocurency':'cryptocurrency',
                'simpliv':'simple','quoras':'quora','schizoids':'psychopath','remainers':'remainder','twinflame':'soulmate','quorans':'quora','brexit':'demonetized',
                'iiest':'institute','dceu':'comics','pessat':'exam','uceed':'college','bhakts':'devotee','boruto':'anime',
                'cryptocoin':'bitcoin','blockchains':'blockchain','fiancee':'fiance','redmi':'smartphone','oneplus':'smartphone','qoura':'quora','deepmind':'framework','ryzen':'cpu','whattsapp':'whatsapp',
                'undertale':'adventure','zenfone':'smartphone','cryptocurencies':'cryptocurrencies','koinex':'bitcoin','zebpay':'bitcoin','binance':'bitcoin','whtsapp':'whatsapp',
                'reactjs':'framework','bittrex':'bitcoin','bitconnect':'bitcoin','bitfinex':'bitcoin','yourquote':'your quote','whyis':'why is','jiophone':'smartphone',
                'dogecoin':'bitcoin','onecoin':'bitcoin','poloniex':'bitcoin','7700k':'cpu','angular2':'framework','segwit2x':'bitcoin','hashflare':'bitcoin','940mx':'gpu',
                'openai':'framework','hashflare':'bitcoin','1050ti':'gpu','nearbuy':'near buy','freebitco':'bitcoin','antminer':'bitcoin','filecoin':'bitcoin','whatapp':'whatsapp',
                'empowr':'empower','1080ti':'gpu','crytocurrency':'cryptocurrency','8700k':'cpu','whatsaap':'whatsapp','g4560':'cpu','payymoney':'pay money',
                'fuckboys':'fuck boys','intenship':'internship','zcash':'bitcoin','demonatisation':'demonetization','narcicist':'narcissist','mastuburation':'masturbation',
                'trignometric':'trigonometric','cryptocurreny':'cryptocurrency','howdid':'how did','crytocurrencies':'cryptocurrencies','phycopath':'psychopath',
                'bytecoin':'bitcoin','possesiveness':'possessiveness','scollege':'college','humanties':'humanities','altacoin':'bitcoin','demonitised':'demonetized',
                'brasília':'brazilia','accolite':'accolyte','econimics':'economics','varrier':'warrier','quroa':'quora','statergy':'strategy','langague':'language',
                'splatoon':'game','7600k':'cpu','gate2018':'gate 2018','in2018':'in 2018','narcassist':'narcissist','jiocoin':'bitcoin','hnlu':'hulu','7300hq':'cpu',
                'weatern':'western','interledger':'blockchain','deplation':'deflation', 'cryptocurrencies':'cryptocurrency', 'bitcoin':'blockchain cryptocurrency',}
#Thay thế các từ sai chính tả
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [ ]:
contraction_dict = {
 "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", 
 "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", 
 "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
 "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
 "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", 
 "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", 
 "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  
 "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
 "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
 "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
 "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
 "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", 
 "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
 "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
 "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", 
 "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
 "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", 
 "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
 "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", 
 "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
 "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
 "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", 
 "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
 "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
 "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
 "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
 "y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", 
 "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
#Thay thế các từ viết tắt
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [ ]:
def clean_sentence(x):
    x = x.lower()
    x = clean_punct(x)
    x = clean_numbers(x)
    x = replace_typical_misspell(x)
#   x = remove_stopwords(x)
    x = replace_contractions(x)
    x = stem_text(x)
    x = lemma_text(x)
    x = x.replace("'","")
    return x

In [ ]:
x['question_text'] = x['question_text'].apply(lambda x: clean_sentence(x))
test['question_text'] = test['question_text'].apply(lambda x: clean_sentence(x))

**3.3. Phân tích các nhãn được trích xuất bằng word cloud**
* Word Cloud là một kỹ thuật trực quan hóa dữ liệu được sử dụng để biểu diễn dữ liệu văn bản, trong đó kích thước của mỗi từ cho biết tần suất hoặc tầm quan trọng của nó. Tạo word cloud để thấy được các từ xuất hiện thường xuyên nhất trong những câu hỏi toxic hay không toxic.
* Các từ có kích thước càng lớn thì tần suất xuất hiện càng nhiều

In [ ]:
def cloud(text, title, size = (10,7)):
    words_list = text.unique().tolist()
    words = ' '.join(words_list)
    
    wordcloud = WordCloud(width=800, height=400,
                          collocations=False
                         ).generate(words)
    
    fig = plt.figure(figsize=size, dpi=80, facecolor='k',edgecolor='k')
    plt.imshow(wordcloud,interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=25,color='w')
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
cloud(train[train['target']==0]['question_text'], 'Sincere Questions On Question_text')

In [ ]:
cloud(x[x['target']==0]['question_text'], 'Sincere Questions On Question_text')

In [ ]:
cloud(train[train['target']==1]['question_text'], 'Insincere Questions On question_text')

In [ ]:
cloud(x[x['target']==1]['question_text'], 'Insincere Questions On question_text')

> Nhận xét :
> * Các câu hỏi insincere thường có nhiều các từ mang nghĩa xấu
> * Tuy nhiên một số các từ không mang nghĩa xấu có tần suất cao như people, will, many, much. Các từ này thuộc stopwords, tức là các từ cần thiết trong ngữ pháp nhưng không mang lại nhiều ý nghĩa khi xét từng từ riêng lẻ.

> Nhận xét :
> * Các câu hỏi insincere thường có nhiều các từ mang nghĩa xấu; các từ về sắc tộc, tôn giáo , giới tính xuất hiện nhiều : white , black , muslim , islam , ....
> * Tuy nhiên một số các từ không mang nghĩa xấu có tần suất cao như people, will, many, much. Các từ này thuộc stopwords, tức là các từ cần thiết trong ngữ pháp nhưng không mang lại nhiều ý nghĩa khi xét từng từ riêng lẻ.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from sklearn.metrics import f1_score

* Chia tập dữ liệu train , test với test_size = 0.2

In [ ]:
train_x, test_x,train_y, test_y = train_test_split(x['question_text'], x['target'], test_size=0.2, random_state=0)
print('x_train: ', train_x.shape, train_y.shape)
print('x_test: ',test_x.shape, test_y.shape)

* **Text Vectorization**

**CountVectorizer**

* Các giải thuật Machine Learning chỉ làm việc được với số, nên sẽ convert text về định dạng số

* Chia câu hỏi thành các từ (đơn vị cơ sở). Cần có một bộ tokenizer có kích thước bằng toàn bộ các từ xuất hiện trong văn bản hoặc bằng toàn bộ các từ có trong từ điển. Một câu văn sẽ được biểu diễn bằng một sparse vector mà mỗi một phần tử đại diện cho một từ, giá trị của nó bằng 0 hoặc 1 tương ứng với từ không xuất hiện hoặc có xuất hiện.

* Sử dụng các túi từ (bags of words) để tạo ra một vector có độ dài bằng độ dài của tokenizer và mỗi phần tử của túi từ sẽ đếm số lần xuất hiện của một từ trong câu và sắp xếp chúng theo một vị trí phù hợp trong vector

* Học trên tập từ vựng của toàn bộ tập train và test, vector đếm có thể phải mã hoá những từ có ở tập test và tập train

* Hạn chế : Các biểu diễn theo túi từ có hạn chế đó là không phân biệt được 2 câu văn có cùng các từ bởi túi từ không phân biệt thứ tự trước sau của các từ trong một câu. Ví dụ như ‘you have no dog’ và ‘no, you have dog’ là 2 câu văn có biểu diễn giống nhau mặc dù có ý nghĩa trái ngược nhau

**Evaluation**

* Đối với dữ liệu mất cân bằng thì sự đánh giá sẽ không tập trung vào Accuracy, thay vào đó ta sẽ tập trung vào điểm F1 , Precision và Recall

* Công thức tính điểm F1 : F1 = 2 (precision recall) / (precision + recall)

* Bằng việc sử dụng sklearn.metrics ta sẽ tính được f1-score , accuracy_score

In [ ]:
vectorizer = CountVectorizer()
bow_train = vectorizer.fit_transform(train_x) 
print(bow_train.shape)
bow_test = vectorizer.transform(test_x)
print(bow_test.shape)
print("Done creating Bag-of-Words")

# **4. Xây dựng mô hình**
**4.1. Logistic Regression**

Link notion: https://excessive-source-1c9.notion.site/18-09-2021-H-i-quy-Logistics-6c3eb44266e549169a0d01cc2d36cfa2

* Mô hình đầu tiên được sử dụng là logistic regression , đây là một thuật toán trong học có giám sát nhằm mục đích phân loại dữ liệu và rất phổ biến cho bài toán phân loại tuyến tính

* Hồi quy logistic là một phương pháp phân tích thống kê được sử dụng để dự đoán giá trị dữ liệu dựa trên các quan sát trước đó của tập dữ liệu.

* Mục đích của hồi quy logistic là ước tính xác suất của các sự kiện, bao gồm xác định mối quan hệ giữa các tính năng từ đó đự đoán xác suất của các kết quả, nên đối với hồi quy logistic ta sẽ có:

> Input: dữ liệu input (ta sẽ coi có hai nhãn là 0 và 1).
> Output : Xác suất dữ liệu input rơi vào nhãn 0 hoặc nhãn 1.

In [ ]:
print(f"Results of logistic regression on full bag-of-words")
logistic = LogisticRegression(penalty="l2", C=1, solver='liblinear') 
logistic.fit(bow_train, train_y) 
train_predictions = logistic.predict(bow_train)
train_acc = accuracy_score(train_y, train_predictions)  
train_f1 = f1_score(train_y, train_predictions) 
print(f"Training accuracy: {train_acc:.2%}, F1: {train_f1:.4f}") 
test_predictions = logistic.predict(bow_test)
test_acc = accuracy_score(test_y, test_predictions) 
test_f1 = f1_score(test_y, test_predictions) 
print(f"Testing accuracy:  {test_acc:.2%}, F1: {test_f1:.4f}")

**4.2. Gradient Boosting**

Notion: https://excessive-source-1c9.notion.site/11-11-2021-Ph-ng-ph-p-Boosting-714c287d19494eb88e8b13cc9d96650e

* Ý tưởng cơ bản của thuật toán Gradient Boosting là lần lượt thêm các decision trees nối tiếp nhau. Tree thêm vào sau sẽ cố gắng giải quyết những sai sót của tree trước đó.

* Xây dựng một lượng lớn các model (thường là cùng loại). Mỗi model sau sẽ học cách sửa những lỗi của model trước (dữ liệu mà model trước dự đoán sai) -> tạo thành một chuỗi các model mà model sau sẽ tốt hơn model trước bởi trọng số được update qua mỗi model (cụ thể ở đây là trọng số của những dữ liệu dự đoán đúng sẽ không đổi, còn trọng số của những dữ liệu dự đoán sai sẽ được tăng thêm) . Chúng ta sẽ lấy kết quả của model cuối cùng trong chuỗi model này làm kết quả trả về.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
print(f"Results of gradient boosting on full bag-of-words")
gbc = GradientBoostingClassifier() 
gbc.fit(bow_train, train_y) 
train_predictions_gbc = gbc.predict(bow_train)
train_acc_gbc = accuracy_score(train_y, train_predictions_gbc) 
train_f1_gbc = f1_score(train_y, train_predictions_gbc) 
print(f"Training accuracy: {train_acc_gbc:.2%}, F1: {train_f1_gbc:.4f}") 
test_predictions_gbc = gbc.predict(bow_test)
test_acc_gbc = accuracy_score(test_y, test_predictions_gbc) 
test_f1_gbc = f1_score(test_y, test_predictions_gbc) 
print(f"Testing accuracy:  {test_acc_gbc:.2%}, F1: {test_f1_gbc:.4f}")

> Nhận xét:
> * F1-score thấp hơn so với Logistic Legression
> * Tỉ lệ accuracy thấp
> * Nhìn vào hiệu suất, ta có thể thấy Logistic Regression hiệu quả hơn trong bài toán này

# Submission

In [ ]:
x_val = vectorizer.transform(test['question_text'])
validation_predictions = logistic.predict(x_val)
submission = pd.DataFrame({'qid':test['qid'], 'prediction':validation_predictions })
submission.to_csv('submission.csv', index=False)
submission